In [ ]:
from llm_evals.checks import ContainsCheck, PythonCodeBlocksPresent
from llm_evals.eval import Eval, PromptTest


eval_obj = Eval(
    test_sequence=PromptTest(
        prompt='Create a python function called `mask_emails` that masks emails',
        checks=[
            ContainsCheck(value='def mask_emails('),
            PythonCodeBlocksPresent(),
        ],
    ),
)
print(eval_obj)

In [ ]:
fake_llm = lambda _: "```python\ndef mask_emails(emails):\n...\n```"  # noqa
result = eval_obj(fake_llm)
print(result)

In [ ]:
result.results

In [ ]:
{'a': [1, 2, 3], 'b': [4, 5, 6]} == {'a': [1, 2, 3], 'b': [4, 5, 6]}

In [ ]:
import yaml
with open('../evals/evals/mask_emails.yaml') as f:
    config = yaml.safe_load(f)

# def mask_emails\([a-zA-Z_]+\: str\) -> str\:
# def mask_emails\\([a-zA-Z_]+\\:?( str)?\\)( -> str)?\\:
pattern = config['test_sequence'][0]['checks'][1]['pattern']

# Function to test if a string matches the given regex pattern
import re
def test_regex(input_string):
    return bool(re.match(pattern, input_string))

# Test strings
test_strings = [
    "def mask_emails(email: str) -> str:",
    "def mask_emails(email: str):",
    "def mask_emails(email) -> str:",
    "def mask_emails(email:str) -> str:",
    "def mask_emails(email) :",
    "def mask_emails_123(email:str):",
    "mask_emails(email:str):",
]

# Testing the regex with different strings
test_results = {string: test_regex(string) for string in test_strings}
test_results

In [ ]:
a = [1, 2, 3]

In [ ]:
print(a)
print(a.pop(0))
print(a)

In [ ]:
{'asdf': 1, 'ffff': 2}.copy().pop('asdf')

In [ ]:
import inspect

def my_function():
    # Your function's code
    print("Hello, world!")

function_as_string = inspect.getsource(my_function)

print(function_as_string)


In [ ]:
sum(x for x in [1, 0, None])

In [ ]:
from typing import ClassVar
from pydantic import BaseModel

class Check(BaseModel):
    type_name: str  # Discriminator field
    _registry: ClassVar[dict] = {}

    @classmethod
    def register_subclass(cls, subclass):
        cls._registry[subclass.type_name] = subclass
        return subclass

    @classmethod
    def deserialize(cls, data: dict):
        type_name = data.get('type_name')
        if type_name in cls._registry:
            return cls._registry[type_name](**data)
        raise ValueError(f"Unknown type {type_name}")


@Check.register_subclass
class XCheck(Check):
    type_name: str = 'xcheck'
    # Add any specific fields and methods for XCheck

@Check.register_subclass
class YCheck(Check):
    type_name: str = 'ycheck'
    # Add any specific fields and methods for YCheck




xcheck_instance = XCheck()
ycheck_instance = YCheck()

# Serialize to dictionary
xcheck_dict = xcheck_instance.dict()
ycheck_dict = ycheck_instance.dict()


In [ ]:
from pydantic import BaseModel
from typing import ClassVar

# class Check(BaseModel):
#     type_name: str  # Discriminator field
#     _registry: ClassVar[dict] = {}

#     @classmethod
#     def register_subclass(cls, type_name):
#         def decorator(subclass):
#             cls._registry[type_name] = subclass
#             return subclass
#         return decorator

#     @classmethod
#     def deserialize(cls, data: dict):
#         type_name = data.get('type_name')
#         if type_name in cls._registry:
#             return cls._registry[type_name](**data)
#         raise ValueError(f"Unknown type {type_name}")

class Check(BaseModel):
    type_name: str  # Discriminator field
    _registry: ClassVar[dict] = {}

    @classmethod
    def register_subclass(cls, type_name):
        def decorator(subclass):
            subclass.type_name = type_name  # Set type_name attribute on subclass
            cls._registry[type_name] = subclass
            return subclass
        return decorator

    @classmethod
    def deserialize(cls, data: dict):
        type_name = data.get('type_name')
        if type_name in cls._registry:
            return cls._registry[type_name](**data)
        raise ValueError(f"Unknown type {type_name}")

In [ ]:
@Check.register_subclass('xcheck')
class XCheck(Check):
    # type_name: str = 'xcheck'
    var: str = 'x'

@Check.register_subclass('ycheck')
class YCheck(Check):
    # type_name: str = 'ycheck'
    var: str = 'y'


In [ ]:
xcheck_instance = XCheck()
ycheck_instance = YCheck()

# # Serialize to dictionary
xcheck_dict = xcheck_instance.model_dump()
print(xcheck_dict)
ycheck_dict = ycheck_instance.model_dump()
print(ycheck_dict)


In [ ]:
Check.deserialize(xcheck_dict)

In [ ]:
Check.deserialize(ycheck_dict)

In [ ]:
from pydantic import BaseModel
from typing import ClassVar

class Check(BaseModel):
    type_name: ClassVar[str]  # Discriminator field as a ClassVar
    _registry: ClassVar[dict] = {}

    @classmethod
    def register_subclass(cls, type_name):
        def decorator(subclass):
            subclass.type_name = type_name  # Set type_name as a class attribute
            cls._registry[type_name] = subclass
            return subclass
        return decorator

    @classmethod
    def deserialize(cls, data: dict):
        type_name = data.get('type_name')
        if type_name in cls._registry:
            return cls._registry[type_name](**data)
        raise ValueError(f"Unknown type {type_name}")

    def serialize(self):
        value = self.model_dump()
        if self.type_name:
            value['type_name'] = self.type_name
        return value


@Check.register_subclass('xcheck')
class XCheck(Check):
    # type_name: str = 'xcheck'
    var: str = 'x'

@Check.register_subclass('ycheck')
class YCheck(Check):
    # type_name: str = 'ycheck'
    var: str = 'y'


xcheck_instance = XCheck()
ycheck_instance = YCheck()

# # Serialize to dictionary
xcheck_dict = xcheck_instance.serialize()
print(xcheck_dict)
ycheck_dict = ycheck_instance.serialize()
print(ycheck_dict)

print(Check.deserialize(xcheck_dict))
print(Check.deserialize(ycheck_dict))


In [ ]:
xcheck_instance.type_name

In [ ]:
from pydantic import BaseModel
from typing import ClassVar

class Check(BaseModel):
    type_name: ClassVar[str | None] = None
    _registry: ClassVar[dict] = {}

    @classmethod
    def register_subclass(cls, type_name: str):  # noqa: ANN102
        """Register a subclass of Check."""
        def decorator(subclass):
            subclass.type_name = type_name  # Set type_name as a class attribute
            cls._registry[type_name] = subclass
            return subclass
        return decorator

    @classmethod
    def from_dict(cls, data: dict):
        type_name = data.get('type_name')
        if type_name in cls._registry:
            return cls._registry[type_name](**data)
        raise ValueError(f"Unknown type {type_name}")

    def to_dict(self):
        value = self.model_dump()
        if self.type_name:
            value['type_name'] = self.type_name
        return value


@Check.register_subclass('xcheck')
class XCheck(Check):
    # type_name: str = 'xcheck'
    var: str = 'x'

@Check.register_subclass('ycheck')
class YCheck(Check):
    # type_name: str = 'ycheck'
    var: str = 'y'

class ZCheck(Check):
    var: str = 'z'


xcheck_instance = XCheck()
ycheck_instance = YCheck()
zcheck_instance = ZCheck()

# # Serialize to dictionary
xcheck_dict = xcheck_instance.to_dict()
print(xcheck_dict)
ycheck_dict = ycheck_instance.to_dict()
print(ycheck_dict)
zcheck_dict = zcheck_instance.to_dict()
print(zcheck_dict)

print(Check.from_dict(xcheck_dict))
print(Check.from_dict(ycheck_dict))
print(Check.from_dict(zcheck_dict))


In [ ]:
num_cpus = 20
candidates = list(range(10))
for i in range(0, len(candidates), num_cpus):
    candidate_batch = candidates[i:i + num_cpus]
    print(candidate_batch)


In [2]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
# base_url = 'http://localhost:1234/v1'
base_url = 'http://host.docker.internal:1234/v1'
client = OpenAI(base_url=base_url, api_key='not-needed')

completion = client.chat.completions.create(
  model='local-model', # this field is currently unused
  messages=[
    {'role': 'system', 'content': "You are a helpful AI assistant."},
    {'role': 'user', 'content': "Write a python function called mask_emails that tasks a string and masks all emails in it."},
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="```python\nimport re\ndef mask_emails(text):\n    pattern = r'\\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Z|a-z]{2,}\\b'\n    emails = re.findall(pattern, text)\n    for email in emails:\n        text = text.replace(email, '{' + str(len(emails)) + '}' + '[EMAIL HIDDEN]')\n    return text\n```\nThis function uses a regular expression pattern to find all emails in the given string and then masks them by replacing each email with a placeholder containing the number of occurrences of email addresses in the string and the phrase '[EMAIL HIDDEN]'. For example, if there are 3 emails in the string, the first email will be masked as '{3}' + '[EMAIL HIDDEN]', the second one as '{2}' + '[EMAIL HIDDEN]' and so on. You can replace '[EMAIL HIDDEN]' with any placeholder or pattern you prefer for hiding actual emails in the output.", role='assistant', function_call=None, tool_calls=None)


In [5]:
from IPython.display import display, Markdown
content = completion.choices[0].message.content
display(Markdown(content))

```python
import re
def mask_emails(text):
    pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    emails = re.findall(pattern, text)
    for email in emails:
        text = text.replace(email, '{' + str(len(emails)) + '}' + '[EMAIL HIDDEN]')
    return text
```
This function uses a regular expression pattern to find all emails in the given string and then masks them by replacing each email with a placeholder containing the number of occurrences of email addresses in the string and the phrase '[EMAIL HIDDEN]'. For example, if there are 3 emails in the string, the first email will be masked as '{3}' + '[EMAIL HIDDEN]', the second one as '{2}' + '[EMAIL HIDDEN]' and so on. You can replace '[EMAIL HIDDEN]' with any placeholder or pattern you prefer for hiding actual emails in the output.

In [ ]:

from llm_evals.eval import EvalResult
import yaml

with open('__temp__/0E454A44-627C-4B56-915B-8DCFE9DF2A55-8C297EA8-42B9-4237-9571-5C95A042E3BA.yaml') as f:
    config = yaml.safe_load(f)

result = EvalResult(**config)

In [ ]:
result.eval_obj.test_sequence[0].ideal_response

In [6]:
from llm_evals.llms.openai import OpenAIServerChat

base_url = 'http://host.docker.internal:1234/v1'
chat = OpenAIServerChat(base_url=base_url, streaming_callback=lambda x: print(x.response, end='', flush=True))  # noqa
response = chat("Write a python function called mask_emails that tasks a string and masks all emails in it.")

```python
import re
def mask_emails(text):
    pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    emails = re.findall(pattern, text)
    for email in emails:
        text = text.replace(email, '{' + str(len(emails)) + '}' + '[EMAIL HIDDEN]')
    return text
```
This function uses a regular expression pattern to find all emails in the given string and then masks them by replacing each email with a placeholder containing the number of occurrences of email addresses in the string and the phrase '[EMAIL HIDDEN]'. For example, if there are 3 emails in the string, the first email will be masked as '{3}' + '[EMAIL HIDDEN]', the second one as '{2}' + '[EMAIL HIDDEN]' and so on. You can replace '[EMAIL HIDDEN]' with any placeholder or pattern you prefer for hiding actual emails in the output.

In [7]:
display(Markdown(response))

```python
import re
def mask_emails(text):
    pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    emails = re.findall(pattern, text)
    for email in emails:
        text = text.replace(email, '{' + str(len(emails)) + '}' + '[EMAIL HIDDEN]')
    return text
```
This function uses a regular expression pattern to find all emails in the given string and then masks them by replacing each email with a placeholder containing the number of occurrences of email addresses in the string and the phrase '[EMAIL HIDDEN]'. For example, if there are 3 emails in the string, the first email will be masked as '{3}' + '[EMAIL HIDDEN]', the second one as '{2}' + '[EMAIL HIDDEN]' and so on. You can replace '[EMAIL HIDDEN]' with any placeholder or pattern you prefer for hiding actual emails in the output.

In [9]:
response = chat("Now write some assertion statements that will test the function.")
display(Markdown(response))

```python
def test_mask_emails():
    text = "John Doe <john.doe@example.com> and Jane Doe <jane.doe2@example.net> are new users."
    expected_output = "John Doe <{1}' + '[EMAIL HIDDEN]'> and Jane Doe <{2}' + '[EMAIL HIDDEN]'> are new users."
    assert mask_emails(text) == expected_output, "Function did not mask emails correctly"
    
    text = "Please contact john.doe@example.com or jane.doe2@example.net for support."
    expected_output = "Please contact {1}' + '[EMAIL HIDDEN]'> or {2}' + '[EMAIL HIDDEN]'> for support."
    assert mask_emails(text) == expected_output, "Function did not mask emails correctly"

test_mask_emails()
```
These test statements check that the `mask_emails` function correctly masks all

 occurrences of email addresses in a given string. The first test statement checks that two emails are correctly masked and replaced with placeholders, while the second test statement checks that multiple occurrences of the same email address are also correctly masked.

```python
def test_mask_emails():
    text = "John Doe <john.doe@example.com> and Jane Doe <jane.doe2@example.net> are new users."
    expected_output = "John Doe <{1}' + '[EMAIL HIDDEN]'> and Jane Doe <{2}' + '[EMAIL HIDDEN]'> are new users."
    assert mask_emails(text) == expected_output, "Function did not mask emails correctly"
    
    text = "Please contact john.doe@example.com or jane.doe2@example.net for support."
    expected_output = "Please contact {1}' + '[EMAIL HIDDEN]'> or {2}' + '[EMAIL HIDDEN]'> for support."
    assert mask_emails(text) == expected_output, "Function did not mask emails correctly"

test_mask_emails()
```
These test statements check that the `mask_emails` function correctly masks all occurrences of email addresses in a given string. The first test statement checks that two emails are correctly masked and replaced with placeholders, while the second test statement checks that multiple occurrences of the same email address are also correctly masked.